In [7]:
import torch
import torch.nn as nn
from Heston_util import *

In [8]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"running on {device}")

running on cuda


In [9]:
sequence_length = 30
dt = 1/365
learning_rate = 0.005
batch_size = 10000

sigma = 2
T = dt * sequence_length
K = 100
s0 = 100
v0 = 0.04
alpha = 1.
b = 0.04
rho = -0.7
alpha_loss = 0.5
transaction_cost_rate = 0.0

In [10]:
def performance(network, S, V, VarPrice):
    network.eval()
    input_vector = torch.cat((torch.log(S[:, :-1]).unsqueeze(-1), V[:, :-1].unsqueeze(-1)), dim=-1)
    holding = network(input_vector).squeeze()
    delta_S = S[:, 1:] - S[:, :-1]
    delta_V = VarPrice[:, 1:] - VarPrice[:, :-1]
    delta_price = torch.cat((delta_S.unsqueeze(-1), delta_V.unsqueeze(-1)), dim=2)
    PnL = (holding * delta_price).sum(dim=(1,2))
    C_T = loss_fn.terminal_payoff(S[:,-1])
    X = -PnL + C_T    
    p0 = find_optimal_p0(X,alpha_loss)
    X = X - p0  
    return (torch.max(X, torch.zeros_like(X)).mean().item()/0.5+p0)

In [11]:
loss_fn = loss_CVAR(Strike_price=K, vol=sigma, T=T, alpha_loss=alpha_loss, trans_cost_rate=transaction_cost_rate, p0_mode='search').to(device=device)

In [12]:
# Load the data
data = torch.load('../Data/Heston_test.pt') # Load the data from the .pt file
N=1000000 # Number of samples
S, V, VarPrice = data[0][:N], data[1][:N], data[2][:N] 
S,V,VarPrice = S.to(device), V.to(device), VarPrice.to(device)

In [13]:
# Load the network
network = RNN_BN_simple(sequence_length=sequence_length).to(device=device)
network.eval()
network.load_state_dict(torch.load('../Result/Heston.pth'))

<All keys matched successfully>

In [14]:
# Evaluate the performance of the network
result = performance(network, S, V, VarPrice)
print(result)

1.9301657378673553
